In [6]:
import cv2
import numpy as np
from ultralytics import YOLO
import supervision as sv
import os

model = YOLO("runs/detect/train4/weights/best.pt")


In [3]:
def read_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

def save_video(output_video_frames, output_video_path):
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(output_video_path, fourcc, 24, (output_video_frames[0].shape[1], output_video_frames[0].shape[0]))
    for frame in output_video_frames:
        out.write(frame)
    out.release()

In [9]:
bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

def render_video(video_path, output_video_path):
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"File {video_path} does not exist")
    frames = read_video(video_path)
    output_video_frames = []
    for frame in frames:
        frame = cv2.resize(frame, (640, 640))
        results = model(frame)[0]
        detections = sv.Detections.from_ultralytics(results)
        labels = [model.model.names[class_id]
                  for class_id in detections.class_id]
        annotated_frame = bounding_box_annotator.annotate(
            scene=frame, detections=detections)
        annotated_frame = label_annotator.annotate(
            scene=annotated_frame, detections=detections, labels=labels
        )
        output_video_frames.append(annotated_frame)
    save_video(output_video_frames, output_video_path)


video_path = "VID20240408175741.mp4"
output_video_path = "output.mp4"
render_video(video_path, output_video_path)

In [2]:
import cv2
import time

cap = cv2.VideoCapture("output.mp4")

# Get the frame rate of the video
fps = cap.get(cv2.CAP_PROP_FPS)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow("frame", frame)
    # Wait for the correct delay between frames
    time.sleep(0.9/ fps)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()